# Making maps with Python {#map-making}

## Introduction

Geopandas explore has been used in previous chapters.

<!-- Decision of whether to use static or interactive. -->
<!-- Flow diagram? -->

## Static maps

### Faceted maps
### Exporting maps as images

<!-- ## Animated maps -->
## Interactive maps

- When are interactive maps useful

### Publishing interactive maps

<!-- ## Mapping applications

Streamlit? -->

## Exercises